In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("serdarakyol/interpress-turkish-news-classification")
model = AutoModelForSequenceClassification.from_pretrained("serdarakyol/interpress-turkish-news-classification")

In [13]:
import torch
from sklearn.metrics import confusion_matrix, classification_report

In [12]:
import platform
platform.platform()

'macOS-13.2-arm64-arm-64bit'

In [14]:
# this ensures that the current MacOS version is at least 12.3+
print(torch.backends.mps.is_available())
# this ensures that the current current PyTorch installation was built with MPS activated.
print(torch.backends.mps.is_built())

True
True


In [15]:
print(torch.has_mps)

True


In [17]:
device = torch.device("mps")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [21]:
import numpy as np

def prediction(news):
    news=[news]
    indices=tokenizer.batch_encode_plus(
    news,
    max_length=512,
    add_special_tokens=True,
    return_attention_mask=True,
    padding='max_length',
    truncation=True,
    return_tensors='pt')
    indices.to(device)
    inputs = indices["input_ids"].clone().detach().to(device)
    masks = indices["attention_mask"].clone().detach().to(device)

    with torch.no_grad():
        output = model(inputs, token_type_ids=None,attention_mask=masks)

    logits = output[0]
    logits = logits.detach().cpu().numpy()
    pred = np.argmax(logits,axis=1)[0]
    return pred

In [22]:
news = r"ABD'den Prens Selman'a yaptırım yok Beyaz Saray Sözcüsü Psaki, Muhammed bin Selman'a yaptırım uygulamamanın \"doğru karar\" olduğunu savundu. Psaki, \"Tarihimizde, Demokrat ve Cumhuriyetçi başkanların yönetimlerinde diplomatik ilişki içinde olduğumuz ülkelerin liderlerine yönelik yaptırım getirilmemiştir\" dedi."


In [23]:
labels = {
    0 : "Culture-Art",
    1 : "Economy",
    2 : "Politics",
    3 : "Education",
    4 : "World",
    5 : "Sport",
    6 : "Technology",
    7 : "Magazine",
    8 : "Health",
    9 : "Agenda"
}
pred = prediction(news)
print(labels[pred])
# > World


World


In [24]:
import pandas as pd
import numpy as np

In [39]:
df = pd.read_excel('Ironic Data/topic_real_values_serdarakyol.xlsx')
df

/Users/tariktalay/Desktop/Thesis/venv/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,title,subject
0,Mert Yuvak(27) Sevgililer Günü Öncesi Kritik V...,Magazine
1,Isparta'yı -5 Derecede 48 Saat Elektriksiz Bır...,Economy
2,Rapor: Mühendislik Eğitimi Artı MBA Artı Özel ...,Economy
3,Normal Takvimle Girdiği 2022'den de Umudu Kese...,Magazine
4,Son 3 Yılda 6 Kez Başkan Değiştiren TÜİK'te İh...,Politics
...,...,...
3166,Takımdan Ayrı Düz Koşu Yapan Futbolcu Ortadan ...,Sport
3167,"Kameraya ""5 İşareti"" Yapan Taraftarlar Stada A...",Sport
3168,"Hentbol Federasyonu'ndan Sert Çıkış: ""Utanılac...",Sport
3169,Deplasman Röportaj Yönetmeliğine Uymayan Futbo...,Sport


In [40]:
df.groupby(["subject"])["subject"].count()

subject
Economy       651
Education     194
Health        182
Magazine      356
Politics      867
Sport         261
Technology    299
World         361
Name: subject, dtype: int64

In [41]:
df['subject'] = df['subject'].str.lower()

In [42]:
df.drop(df[df['subject']=='magazine'].index, inplace=True)
df

,title,subject
1,Isparta'yı -5 Derecede 48 Saat Elektriksiz Bır...,economy
2,Rapor: Mühendislik Eğitimi Artı MBA Artı Özel ...,economy
4,Son 3 Yılda 6 Kez Başkan Değiştiren TÜİK'te İh...,politics
5,Ekonomide Düşündüren Tablo: 1 Haftadır Evde Ya...,economy
6,''Teknik sorunlar'' Nedeniyle Türkiye'ye Doğal...,economy
...,...,...
3166,Takımdan Ayrı Düz Koşu Yapan Futbolcu Ortadan ...,sport
3167,"Kameraya ""5 İşareti"" Yapan Taraftarlar Stada A...",sport
3168,"Hentbol Federasyonu'ndan Sert Çıkış: ""Utanılac...",sport
3169,Deplasman Röportaj Yönetmeliğine Uymayan Futbo...,sport


In [43]:
df.groupby(["subject"])["subject"].count()

subject
economy       651
education     194
health        182
politics      867
sport         261
technology    299
world         361
Name: subject, dtype: int64

In [44]:
df["pred_subject"] = df.apply(lambda x: labels[prediction(x.title)], axis=1)

In [45]:
df

,title,subject,pred_subject
1,Isparta'yı -5 Derecede 48 Saat Elektriksiz Bır...,economy,Economy
2,Rapor: Mühendislik Eğitimi Artı MBA Artı Özel ...,economy,Economy
4,Son 3 Yılda 6 Kez Başkan Değiştiren TÜİK'te İh...,politics,Economy
5,Ekonomide Düşündüren Tablo: 1 Haftadır Evde Ya...,economy,Economy
6,''Teknik sorunlar'' Nedeniyle Türkiye'ye Doğal...,economy,Economy
...,...,...,...
3166,Takımdan Ayrı Düz Koşu Yapan Futbolcu Ortadan ...,sport,Sport
3167,"Kameraya ""5 İşareti"" Yapan Taraftarlar Stada A...",sport,Sport
3168,"Hentbol Federasyonu'ndan Sert Çıkış: ""Utanılac...",sport,Sport
3169,Deplasman Röportaj Yönetmeliğine Uymayan Futbo...,sport,Sport


In [46]:
df.groupby(["pred_subject"])["pred_subject"].count()

pred_subject
Economy       591
Education     177
Health        213
Magazine        9
Politics      689
Sport         375
Technology    391
World         370
Name: pred_subject, dtype: int64

In [47]:
df.drop(df[df['pred_subject']=='magazine'].index, inplace=True)
df

,title,subject,pred_subject
1,Isparta'yı -5 Derecede 48 Saat Elektriksiz Bır...,economy,Economy
2,Rapor: Mühendislik Eğitimi Artı MBA Artı Özel ...,economy,Economy
4,Son 3 Yılda 6 Kez Başkan Değiştiren TÜİK'te İh...,politics,Economy
5,Ekonomide Düşündüren Tablo: 1 Haftadır Evde Ya...,economy,Economy
6,''Teknik sorunlar'' Nedeniyle Türkiye'ye Doğal...,economy,Economy
...,...,...,...
3166,Takımdan Ayrı Düz Koşu Yapan Futbolcu Ortadan ...,sport,Sport
3167,"Kameraya ""5 İşareti"" Yapan Taraftarlar Stada A...",sport,Sport
3168,"Hentbol Federasyonu'ndan Sert Çıkış: ""Utanılac...",sport,Sport
3169,Deplasman Röportaj Yönetmeliğine Uymayan Futbo...,sport,Sport


In [48]:
df['pred_subject'] = df['pred_subject'].str.lower()

In [49]:
df

,title,subject,pred_subject
1,Isparta'yı -5 Derecede 48 Saat Elektriksiz Bır...,economy,economy
2,Rapor: Mühendislik Eğitimi Artı MBA Artı Özel ...,economy,economy
4,Son 3 Yılda 6 Kez Başkan Değiştiren TÜİK'te İh...,politics,economy
5,Ekonomide Düşündüren Tablo: 1 Haftadır Evde Ya...,economy,economy
6,''Teknik sorunlar'' Nedeniyle Türkiye'ye Doğal...,economy,economy
...,...,...,...
3166,Takımdan Ayrı Düz Koşu Yapan Futbolcu Ortadan ...,sport,sport
3167,"Kameraya ""5 İşareti"" Yapan Taraftarlar Stada A...",sport,sport
3168,"Hentbol Federasyonu'ndan Sert Çıkış: ""Utanılac...",sport,sport
3169,Deplasman Röportaj Yönetmeliğine Uymayan Futbo...,sport,sport


In [50]:
print(classification_report(y_pred=df.pred_subject,y_true=df.subject))

              precision    recall  f1-score   support

     economy       0.90      0.82      0.86       651
   education       0.89      0.81      0.85       194
      health       0.69      0.81      0.74       182
    magazine       0.00      0.00      0.00         0
    politics       0.96      0.76      0.85       867
       sport       0.69      1.00      0.82       261
  technology       0.73      0.96      0.83       299
       world       0.91      0.93      0.92       361

    accuracy                           0.84      2815
   macro avg       0.72      0.76      0.73      2815
weighted avg       0.87      0.84      0.85      2815



/Users/tariktalay/Desktop/Thesis/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/tariktalay/Desktop/Thesis/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/tariktalay/Desktop/Thesis/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Topic Modelling with pre-processing

In [29]:
import nltk
import re

In [30]:
from TurkishStemmer import TurkishStemmer
stemmer = TurkishStemmer()
stemmer.stem("bırakanlar")

'bırakan'

In [31]:
stopwordSet = set(nltk.corpus.stopwords.words('turkish'))

In [34]:
def leadMyWord(text):
    text = re.sub('[^a-zA-ZğĞüÜşŞıİöÖçÇ]'," ",text)
    text = re.sub(r'\s+', ' ', text)
    text = text.lower()
    text = nltk.word_tokenize(text,language='turkish')
    text = [word for word in text if not word in stopwordSet]
    #text = [stemmer.stem(i) for i in text]
    text = " ".join(text)
    return text

In [35]:
smp_text = leadMyWord("Mert Yuvak(27) Sevgililer Günü Öncesi Kritik Virajda: Hediye mi Alsam Japon Kirpi Balığı’nın Dişisine Kur Yapma Dansını mı Öğrensem?")
smp_text

'mert yuvak sevgi gün önce kri viraj hedi mi al japon kirp balık nın dişi kur yapma dans öğren'

In [37]:
df["title_preprocess"] = df.apply(lambda x: leadMyWord(x.title),axis=1)
df

,title,subject,pred_subject,title_preprocess
1,Isparta'yı -5 Derecede 48 Saat Elektriksiz Bır...,economy,economy,ispar yı derece saat elektriks bırakan akde el...
2,Rapor: Mühendislik Eğitimi Artı MBA Artı Özel ...,economy,economy,rapor mühendislik eğit art mba art özel sektör...
4,Son 3 Yılda 6 Kez Başkan Değiştiren TÜİK'te İh...,politics,economy,son yıl başkan değiştiren tüi̇k te i̇hale sist...
5,Ekonomide Düşündüren Tablo: 1 Haftadır Evde Ya...,economy,economy,ekonomi düşündüren tablo haf ev ya oksijen i̇s...
6,''Teknik sorunlar'' Nedeniyle Türkiye'ye Doğal...,economy,economy,teknik sor neden türki ye doğalgaz satış durdu...
...,...,...,...,...
3166,Takımdan Ayrı Düz Koşu Yapan Futbolcu Ortadan ...,sport,sport,takım ayr düz koş yapan futbolç orta kaybol
3167,"Kameraya ""5 İşareti"" Yapan Taraftarlar Stada A...",sport,sport,kamera i̇şareti yapan taraftar sta alınmayacak
3168,"Hentbol Federasyonu'ndan Sert Çıkış: ""Utanılac...",sport,sport,hentbol federasyon ndan sert çıkış utanılacak ...
3169,Deplasman Röportaj Yönetmeliğine Uymayan Futbo...,sport,sport,deplasman röportaj yönetmelik uymayan futbolcu...


In [38]:
df["pred-process"] = df.apply(lambda x: labels[prediction(x.title_preprocess)], axis=1)

In [41]:
df.groupby(["pred-process"])["pred-process"].count()

pred-process
Culture-Art    1880
Economy          23
Education         2
Health            4
Magazine        777
Politics         50
Sport            45
Technology       17
World             8
Name: pred-process, dtype: int64

In [46]:
def leadMyWord(text):
    text = re.sub('[^a-zA-ZğĞüÜşŞıİöÖçÇ]'," ",text)
    text = re.sub(r'\s+', ' ', text)
    text = text.lower()
    text = nltk.word_tokenize(text,language='turkish')
    text = [word for word in text if not word in stopwordSet]
    #text = [stemmer.stem(i) for i in text]
    text = " ".join(text)
    return text

In [47]:
smp_text = leadMyWord("Mert Yuvak(27) Sevgililer Günü Öncesi Kritik Virajda: Hediye mi Alsam Japon Kirpi Balığı’nın Dişisine Kur Yapma Dansını mı Öğrensem?")
smp_text

'mert yuvak sevgililer günü öncesi kritik virajda hediye mi alsam japon kirpi balığı nın dişisine kur yapma dansını öğrensem'

In [48]:
df["title_preprocess2"] = df.apply(lambda x: leadMyWord(x.title),axis=1)
df

,title,subject,pred_subject,title_preprocess,pred-process,title_preprocess2
1,Isparta'yı -5 Derecede 48 Saat Elektriksiz Bır...,economy,economy,ispar yı derece saat elektriks bırakan akde el...,Culture-Art,isparta yı derecede saat elektriksiz bırakan a...
2,Rapor: Mühendislik Eğitimi Artı MBA Artı Özel ...,economy,economy,rapor mühendislik eğit art mba art özel sektör...,Culture-Art,rapor mühendislik eğitimi artı mba artı özel s...
4,Son 3 Yılda 6 Kez Başkan Değiştiren TÜİK'te İh...,politics,economy,son yıl başkan değiştiren tüi̇k te i̇hale sist...,Culture-Art,son yılda başkan değiştiren tüi̇k te i̇hale si...
5,Ekonomide Düşündüren Tablo: 1 Haftadır Evde Ya...,economy,economy,ekonomi düşündüren tablo haf ev ya oksijen i̇s...,Culture-Art,ekonomide düşündüren tablo haftadır evde yatan...
6,''Teknik sorunlar'' Nedeniyle Türkiye'ye Doğal...,economy,economy,teknik sor neden türki ye doğalgaz satış durdu...,Magazine,teknik sorunlar nedeniyle türkiye ye doğalgaz ...
...,...,...,...,...,...,...
3166,Takımdan Ayrı Düz Koşu Yapan Futbolcu Ortadan ...,sport,sport,takım ayr düz koş yapan futbolç orta kaybol,Culture-Art,takımdan ayrı düz koşu yapan futbolcu ortadan ...
3167,"Kameraya ""5 İşareti"" Yapan Taraftarlar Stada A...",sport,sport,kamera i̇şareti yapan taraftar sta alınmayacak,Sport,kameraya i̇şareti yapan taraftarlar stada alın...
3168,"Hentbol Federasyonu'ndan Sert Çıkış: ""Utanılac...",sport,sport,hentbol federasyon ndan sert çıkış utanılacak ...,Sport,hentbol federasyonu ndan sert çıkış utanılacak...
3169,Deplasman Röportaj Yönetmeliğine Uymayan Futbo...,sport,sport,deplasman röportaj yönetmelik uymayan futbolcu...,Sport,deplasman röportaj yönetmeliğine uymayan futbo...


In [49]:
df["pred-process2"] = df.apply(lambda x: labels[prediction(x.title_preprocess2)], axis=1)

In [50]:
df.groupby(["pred-process2"])["pred-process2"].count()

pred-process2
Culture-Art    2155
Economy          31
Education         3
Health            4
Magazine        487
Politics         41
Sport            54
Technology       22
World             9
Name: pred-process2, dtype: int64

In [60]:
df[df['subject']=='world']

,title,subject,pred_subject,title_preprocess,pred-process,title_preprocess2,pred-process2
18,"Bulgaristan İçişleri Bakanlığı, +90 Ülke Kodlu...",world,world,bulgaristan i̇çişleri bakanlık ülk kot numara ...,Culture-Art,bulgaristan i̇çişleri bakanlığı ülke kodlu num...,Culture-Art
67,20 Yıl Süren Kanlı Çatışmaların Ardından Afgan...,world,world,yıl süren kanl çatışma art afganistan yönet el...,Culture-Art,yıl süren kanlı çatışmaların ardından afganist...,Culture-Art
68,Taliban Yeni Koronavirüs Tedbirlerini Açıkladı...,world,world,taliban yen koronavirüs tedbir açık kadın dışa...,Magazine,taliban yeni koronavirüs tedbirlerini açıkladı...,Culture-Art
71,Yunanistan'a Yardım İçin 2 Adet Yangın Söndürm...,world,world,yunanistan a yardım i̇çin adet yang söndür uça...,Culture-Art,yunanistan a yardım i̇çin adet yangın söndürme...,Culture-Art
72,Efsanelerde Adı Geçen Dingo'nun Ahırı'nın Türk...,world,world,efsane ad geçen dingo nun ahır nın türki sınır...,Culture-Art,efsanelerde adı geçen dingo nun ahırı nın türk...,Culture-Art
...,...,...,...,...,...,...,...
2915,İngiliz Kraliyet Ailesi'ni Sarsan Gelişme,world,world,i̇ngiliz kraliyet aile ni sar geliş,Culture-Art,i̇ngiliz kraliyet ailesi ni sarsan gelişme,Culture-Art
2916,İsrail'e Diplomatik Misilleme Girişimi Boy Eng...,world,world,i̇srail e diplomatik misille giriş boy engel t...,Culture-Art,i̇srail e diplomatik misilleme girişimi boy en...,Culture-Art
2940,Töre Cinayetleri Brezilya'yı Sarsmaya Devam Ed...,world,world,tör cinayet brezilya yı sarsma devam ediyor,Magazine,töre cinayetleri brezilya yı sarsmaya devam ed...,Culture-Art
2950,Ermeni Diasporası'ndan Tarihi Geri Adım,world,world,ermen diaspora ndan tarih ger ad,Economy,ermeni diasporası ndan tarihi geri adım,Culture-Art


In [61]:
df.to_excel('re_topic_modelled.xlsx')

## savasy Model

In [6]:
df = pd.read_excel('re_topic_modelled.xlsx')
df.drop('Unnamed: 0',axis=1, inplace=True)
df

,title,subject,pred_subject,title_preprocess,pred-process,title_preprocess2,pred-process2
0,Isparta'yı -5 Derecede 48 Saat Elektriksiz Bır...,economy,economy,ispar yı derece saat elektriks bırakan akde el...,Culture-Art,isparta yı derecede saat elektriksiz bırakan a...,Culture-Art
1,Rapor: Mühendislik Eğitimi Artı MBA Artı Özel ...,economy,economy,rapor mühendislik eğit art mba art özel sektör...,Culture-Art,rapor mühendislik eğitimi artı mba artı özel s...,Culture-Art
2,Son 3 Yılda 6 Kez Başkan Değiştiren TÜİK'te İh...,politics,economy,son yıl başkan değiştiren tüi̇k te i̇hale sist...,Culture-Art,son yılda başkan değiştiren tüi̇k te i̇hale si...,Culture-Art
3,Ekonomide Düşündüren Tablo: 1 Haftadır Evde Ya...,economy,economy,ekonomi düşündüren tablo haf ev ya oksijen i̇s...,Culture-Art,ekonomide düşündüren tablo haftadır evde yatan...,Culture-Art
4,''Teknik sorunlar'' Nedeniyle Türkiye'ye Doğal...,economy,economy,teknik sor neden türki ye doğalgaz satış durdu...,Magazine,teknik sorunlar nedeniyle türkiye ye doğalgaz ...,Culture-Art
...,...,...,...,...,...,...,...
2801,Takımdan Ayrı Düz Koşu Yapan Futbolcu Ortadan ...,sport,sport,takım ayr düz koş yapan futbolç orta kaybol,Culture-Art,takımdan ayrı düz koşu yapan futbolcu ortadan ...,Sport
2802,"Kameraya ""5 İşareti"" Yapan Taraftarlar Stada A...",sport,sport,kamera i̇şareti yapan taraftar sta alınmayacak,Sport,kameraya i̇şareti yapan taraftarlar stada alın...,Sport
2803,"Hentbol Federasyonu'ndan Sert Çıkış: ""Utanılac...",sport,sport,hentbol federasyon ndan sert çıkış utanılacak ...,Sport,hentbol federasyonu ndan sert çıkış utanılacak...,Sport
2804,Deplasman Röportaj Yönetmeliğine Uymayan Futbo...,sport,sport,deplasman röportaj yönetmelik uymayan futbolcu...,Sport,deplasman röportaj yönetmeliğine uymayan futbo...,Sport


In [7]:
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer, AutoModelForSequenceClassification
tokenizer= AutoTokenizer.from_pretrained("savasy/bert-turkish-text-classification")
model= AutoModelForSequenceClassification.from_pretrained("savasy/bert-turkish-text-classification")

In [8]:
nlp=pipeline("sentiment-analysis", model=model, tokenizer=tokenizer,)

In [9]:
nlp("Anayasa Mahkemesi, 2000 ve Sonrası Doğumluların Gençliklerinin İptali İstemiyle Yapılan Başvuruyu İncelemeye Aldı...")[0]['label']

'politics'

In [10]:
code_to_label={
 'LABEL_0': 'dunya ',
 'LABEL_1': 'ekonomi ',
 'LABEL_2': 'kultur ',
 'LABEL_3': 'saglik ',
 'LABEL_4': 'siyaset ',
 'LABEL_5': 'spor ',
 'LABEL_6': 'teknoloji '}

code_to_label[nlp("Anayasa Mahkemesi, 2000 ve Sonrası Doğumluların Gençliklerinin İptali İstemiyle Yapılan Başvuruyu İncelemeye Aldı...")[0]['label']]

KeyError: 'politics'

In [11]:
df["subject_savasy"] = df[df['subject']!='education'].apply(lambda x: nlp(x.title)[0]['label'], axis=1)

In [12]:
df = df[['title',
 'subject',
 'pred_subject',
    'subject_savasy']]

In [13]:
df

,title,subject,pred_subject,subject_savasy
0,Isparta'yı -5 Derecede 48 Saat Elektriksiz Bır...,economy,economy,economy
1,Rapor: Mühendislik Eğitimi Artı MBA Artı Özel ...,economy,economy,economy
2,Son 3 Yılda 6 Kez Başkan Değiştiren TÜİK'te İh...,politics,economy,economy
3,Ekonomide Düşündüren Tablo: 1 Haftadır Evde Ya...,economy,economy,economy
4,''Teknik sorunlar'' Nedeniyle Türkiye'ye Doğal...,economy,economy,economy
...,...,...,...,...
2801,Takımdan Ayrı Düz Koşu Yapan Futbolcu Ortadan ...,sport,sport,sport
2802,"Kameraya ""5 İşareti"" Yapan Taraftarlar Stada A...",sport,sport,sport
2803,"Hentbol Federasyonu'ndan Sert Çıkış: ""Utanılac...",sport,sport,sport
2804,Deplasman Röportaj Yönetmeliğine Uymayan Futbo...,sport,sport,sport


In [15]:
print(classification_report(y_pred=df[df['subject']!='education'].subject_savasy,y_true=df[df['subject']!='education'].subject))

              precision    recall  f1-score   support

     culture       0.00      0.00      0.00         0
     economy       0.80      0.78      0.79       650
      health       0.70      0.60      0.65       182
    politics       0.88      0.67      0.76       861
       sport       0.91      0.80      0.85       260
  technology       0.72      0.68      0.70       299
       world       0.65      0.76      0.70       361

    accuracy                           0.72      2613
   macro avg       0.66      0.61      0.63      2613
weighted avg       0.80      0.72      0.75      2613



C:\Users\tarik\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\tarik\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\tarik\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [16]:
df.groupby(['subject_savasy'])['subject_savasy'].count()

subject_savasy
culture       235
economy       634
health        158
politics      652
sport         227
technology    282
world         425
Name: subject_savasy, dtype: int64

In [28]:
df.to_excel('e_topic_modelled2.xlsx')